In [1]:
import time
import tushare as ts
import datetime
import pandas as pd
import os
import tensorflow as tf
from collections import namedtuple
import numpy as np

In [2]:
if os.path.exists('601318.SH_5min.csv'):
    data = pd.read_csv('601318.SH_5min.csv', index_col=0)
    print('Read sussessful')
#     print('head: \n', data.head())
#     print('tail: \n', data.tail())
else:
    print('File not exist')

Read sussessful


In [3]:
data.head(5)

,ts_code,trade_time,open,close,high,low,vol,amount
0,601318.SH,2019-10-22 15:00:00,90.15,90.29,90.29,90.15,966545.0,87224082.0
1,601318.SH,2019-10-22 14:55:00,90.12,90.15,90.19,90.11,735530.0,66309263.0
2,601318.SH,2019-10-22 14:50:00,90.10,90.11,90.12,90.07,558033.0,50276229.0
3,601318.SH,2019-10-22 14:45:00,90.10,90.10,90.14,90.06,640738.0,57733317.0
4,601318.SH,2019-10-22 14:40:00,90.25,90.12,90.25,90.06,724000.0,65254508.0


In [4]:
def get_time(t):
    time = datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    minus_9_30 = (int(time.strftime('%H'))-9) * 12 + int(time.strftime('%M')) / 5 - 6
    return minus_9_30 if minus_9_30 < 25 else minus_9_30 - 18

class Observations:
    def __init__(self, index, is_hold, wait_time, trade_price):
        # is_hold: 是否持有股票，1表示持有，0表示未持有
        # trade_price: 距离上次操作的时间（多少个5分钟）
        # trade_price: 上次交易价格
        
        self.index = index
        self.is_hold = is_hold 
        self.wait_time = wait_time
        self.trade_price = trade_price

    def values(self, history_data, length):
        # history_data: DataFram 索引越靠前日期越靠后
        # 返回数据为 length * 6 + 3， 前 length * 6 为每日的 time，open， close， high， low， vol-10000
        # 其中 time 为 0 到 48， 表示一天中的第几个5分钟
        # 最后三位分别是 is_hold * 100，即100为持仓, 持仓是否过夜，100为过夜
        
        recent_data = history_data[['trade_time', 'open', 'high', 'low', 'close', 'vol']][
            self.index: self.index+length]
        recent_data['vol'] = recent_data['vol']/10000
        recent_data['trade_time'] = recent_data['trade_time'].apply(lambda x: get_time(x))
        is_pass_night = self.wait_time > 48 or self.wait_time > recent_data['trade_time'].iloc[0]
        return np.hstack([np.array(recent_data.values).reshape(1,-1),
                          np.array([[self.is_hold*100, 100 if is_pass_night else 0, self.trade_price]])])
        
    def decode(self, history_data, length, log=False):
        recent_data = history_data[['trade_time', 'open', 'high', 'low', 'close', 'vol']][
            self.index: self.index+length]
        recent_data['trade_time'] = recent_data['trade_time'].apply(lambda x: get_time(x))

        if log:
            print('recent data is :\n', recent_data)
            print('')

            if self.is_hold:
                print('Hold stock for {} minutes， purchase price is {}.'.format(
                    self.wait_time*5, self.trade_price))
            else:
                print('Dosen\'t hold any thing.')
        return recent_data
        
    def __str__(self):
        return 'index: {}, is_hold: {}, wait_time: {}, trade_price: {}\n'.format(
            self.index, self.is_hold, self.wait_time, self.trade_price)
    
    def __repr__(self):
        return self.__str__()

In [ ]:
# Observation test
# index， is_hold， wait_time， trade_price
# obs = Observations(*[1, 1, 0, 89])
# next_obs = Observations(*[0, 1, 1, 89])
# print(obs.values(data, 3))
# print(next_obs.values(data, 3))

In [ ]:
class Actions:
    
    def __init__(self, action_prob):
        # 买、卖、持有的几率
        self.p_buy = action_prob[0]
        self.p_sell = action_prob[1] 
        self.p_hold = 1 - action_prob[0] - action_prob[1] 
        
        self.action_choose = np.random.choice(['buy', 'sell', 'hold'],
                                              p=[self.p_buy, self.p_sell, self.p_hold])
# Deubg 
        print('action_prob: {}, action is {}'.format(action_prob, self.action_choose))
    
    def choose(self):

        return self.action_choose

+ 注意： 交易默认只买 100 股，手续费默认

In [ ]:
def calc_reward_batch(obs, next_obs, history_data):
    # obs 和 next_obs 为 Observation 类
    
    fee = obs.trade_price * 0.02 if next_obs.wait_time == 1 else 0
    if obs.is_hold == 1:
        delta_price = (next_obs.decode(history_data, 1).close.iloc[0]
                       - obs.decode(history_data, 1).close.iloc[0]) * 100
        return delta_price - fee
    else:
        return -fee
        

In [ ]:
# 计算 reward 测试
# obs.decode(data, 3, log=True)
# next_obs.decode(data, 3, log=True)
# calc_reward_batch(obs, next_obs, data)

In [ ]:
class Env:
    def __init__(self, hps, history_data):
        self._hps = hps
        self._history_data = history_data
        
        self._observations_dim = hps.days * 6 + 3
        self._actions_dim = 3
        return
    
    def reset(self):
        index = self._history_data.shape[0] - self._hps.days - 1
        return Observations(index=index, is_hold=0, wait_time=0, trade_price=0)
    
    def step(self, obs, action):
        # 输入为 Observations 类和 Actions 类
        # 返回值为 next observations， reward， done
        index, is_hold, wait_time, trade_price = obs.index, obs.is_hold, obs.wait_time, obs.trade_price
        done = True if index == 0 else False
        action_choose = action.choose()
        
        if is_hold == 1 and action_choose == 'sell':
            current_time = get_time(self._history_data['trade_time'].iloc[index])
            is_pass_night = wait_time>48 or wait_time>current_time
            if is_pass_night:
                is_hold = 0  # 卖掉了
                wait_time = 0  # 时间清0
                trade_price = self._history_data['close'].iloc[index]  # 以当前的收盘价为成交加个
            else:
                pass # 不做操作，类似 hold
        elif is_hold == 0 and action_choose == 'buy':
            is_hold = 1
            wait_time = 0
            trade_price = self._history_data['close'].iloc[index]
        else:
            pass # 不做操作
        
        next_obs = Observations(index-1, is_hold, wait_time+1, trade_price)

        return next_obs, calc_reward_batch(obs, next_obs, self._history_data), done
    
    @property
    def observations_dim(self):
        return self._observations_dim
    
    @property
    def actions_dim(self):
        return self._actions_dim

In [ ]:
class DataSet:
    def __init__(self, hps, history_data):
        self._buffer = []
        self._length = 0
        self._hps = hps
        self._history_data = history_data
        return 
    
    def get_batch(self, nums):
        assert self._length > 1, 'Length of data is {} which is not enough. \
        Data need at least {}'.format(self._length, 2)
        
        rand_idx = np.random.randint(0, self._length-1, nums)
        obs = np.vstack([self._buffer[x].values(
            self._history_data, self._hps.days) for x in rand_idx])
        
        next_obs = np.vstack([self._buffer[x+1].values(
            self._history_data, self._hps.days) for x in rand_idx])
        
        reward = np.array([calc_reward_batch(self._buffer[x], self._buffer[x+1], self._history_data)
                           for x in rand_idx])
    
        return obs, next_obs, reward
    
    def add_data(self, obs):
        # obs 为 Observation 类
        self._buffer.append(obs)
        self._length += 1
        return
        

In [ ]:
# DataSet test

# obs = Observations(*[1, 1, 0, 89])
# next_obs = Observations(*[0, 1, 1, 89])
# data_set = DataSet(hps, data)
# data_set.add_data(obs)
# data_set.add_data(obs)
# data_set.add_data(obs)
# print(data_set._buffer)
# obs.trade_price = 100
# print(data_set._buffer)
# print(obs)
# data_set.add_data(obs)
# obs, next_obs, reward = data_set.get_batch(2)
# print('obs: \n', obs)
# print('\nnext_obs: \n', next_obs)
# print('\nreward: \n', reward)

In [ ]:
class Model:
    def __init__(self, env, hps):
        self._env = env
        self._hps = hps
        
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.action, self.Q, self.action_loss, self.Q_loss, self.action_train_opt, self.Q_train_opt = \
            self._build_graph()
        self._sess, self._summary_writer = self._sess_setup()
        return
    
    def train(self, iteration, data_set):
        for i in range(iteration):
            obs, next_obs, reward = data_set.get_batch(self._hps.batch_size)
            action_loss, Q_loss = self._train_one_step(obs, next_obs, reward)
            print('action_loss: {}, Q_loss : {}'.format(action_loss, Q_loss))

        return
    
    def test(self, data):
        return
    
    def predict(self, obs):
        action_prob = self._sess.run(self.action, {self._observations_ph: obs})
        return action_prob
    
    def _train_one_step(self, obs, next_obs, reward):
        feed_dict = {self._observations_ph: obs,
                    self._next_observations_ph: next_obs,
                    self._rewards_ph: reward}
        
        action_loss, Q_loss, _, _ = self._sess.run([self.action_loss, self.Q_loss,
                                                    self.action_train_opt, self.Q_train_opt],
                                                   feed_dict)
        return action_loss, Q_loss
    
    def _sess_setup(self):
        saver = tf.train.Saver(max_to_keep=3)
        sv = tf.train.Supervisor(logdir=self._hps.train_dir,
                   is_chief=True,
                   saver=saver,
                   summary_op=None,
                   save_summaries_secs=600, # save summaries for tensorboard every 60 secs
                   save_model_secs=600, # checkpoint every 600 secs
                   global_step=self.global_step,
                   init_feed_dict= None
                   )
        summary_writer = sv.summary_writer
        sess = sv.prepare_or_wait_for_session()
    
        return sess, summary_writer
    
    def _create_placeholders(self):
        observations_dim = self._env.observations_dim
        actions_dim = self._env.actions_dim

        
        self._observations_ph = tf.placeholder(
            tf.float32,
            shape=(None, observations_dim),
            name='observation',
        )
        self._next_observations_ph = tf.placeholder(
            tf.float32,
            shape=(None, observations_dim),
            name='next_observation',
        )
#         self._actions_ph = tf.placeholder(
#             tf.float32,
#             shape=(None, actions_dim),
#             name='actions',
#         )
        self._rewards_ph = tf.placeholder(
            tf.float32,
            shape=(None, ),
            name='rewards',
        )
        return
    
#     def _linear(self, arg, output_size, activation, scope=None, reuse=False):
#         input_size = arg.get_shape().as_list()[1]
#         print('input_size', input_size)
#         trunc_norm_init = tf.truncated_normal_initializer(stddev=self._hps.trunc_norm_init_std)
        
#         with tf.variable_scope(scope or "Linear", reuse=reuse):
#             matrix = tf.get_variable("Matrix", [input_size, output_size])
#             res = tf.matmul(arg, matrix)
#             bias_term = tf.get_variable("Bias", [output_size],
#                                         initializer=trunc_norm_init)
#         return activation(res + bias_term)
        
    def _action_Q_output(self, state, reuse=False):
        
        with tf.variable_scope('hidden_state', reuse=tf.AUTO_REUSE):
            hidden_states = tf.layers.dense(state, self._hps.hidden_dim,
                                           activation=tf.nn.sigmoid, name='state_hidden_layer')
            
        with tf.variable_scope('action_output', reuse=tf.AUTO_REUSE):
            actions = tf.nn.softmax(tf.layers.dense(hidden_states, self._env.actions_dim,
                                                    activation=tf.nn.sigmoid, name='action_output_layer'))
            
        with tf.variable_scope('Q_output', reuse=tf.AUTO_REUSE):
            Q = tf.layers.dense(tf.concat([hidden_states, actions], axis=1), 1,
                                           activation=None, name='Q_output_layer')
        return hidden_states, actions, Q
        
    def _build_graph(self):
        self._create_placeholders()
        _, action, Q = self._action_Q_output(self._observations_ph)
        _, _, next_Q = self._action_Q_output(self._next_observations_ph)
        
        # Calculate action loss and Q loss
        action_loss = -tf.reduce_sum(tf.squeeze(Q), axis=0)

        Q_loss = tf.reduce_sum(self._rewards_ph + tf.squeeze(self._hps.gamma * next_Q - Q), axis=0)

        
        # Get update option
        t_vars = tf.trainable_variables()
        action_vars = [var for var in t_vars 
                       if var.name.startswith('hidden_state') or var.name.startswith('action_output')]
        
        Q_vars = [var for var in t_vars 
                  if var.name.startswith('hidden_state') or var.name.startswith('Q_output')] 
        
        action_train_opt = tf.train.AdamOptimizer(self._hps.learning_rate).minimize(
            action_loss, var_list = action_vars)
        
        Q_train_opt = tf.train.AdamOptimizer(self._hps.learning_rate).minimize(
            action_loss, var_list = Q_vars)
        
        return action, Q, action_loss, Q_loss, action_train_opt, Q_train_opt

        

In [ ]:
class Agent:
    def __init__(self, hps, env, history_data):
        self._hps = hps
        self._env = env
        self._history_data = history_data
        self._data_set = DataSet(hps, history_data)
        self._model = Model(env, hps)
        return
    
    def step(self, obs):
        self._data_set.add_data(obs)
        action_prob = self._model.predict(obs.values(self._history_data, self._hps.days))
        action = Actions(action_prob[0])
        
        if self._data_set._length > 20:
            self._model.train(1, self._data_set)
            
        return action
    

In [ ]:
hps = {'trunc_norm_init_std': 1e-4,
      'hidden_dim': 20,
      'train_dir': './model',
      'gamma': 0.99,
      'learning_rate': 0.003,
      'batch_size': 10,
      'days': 20}
hps = namedtuple("HParams", hps.keys())(**hps)

In [ ]:
# For model test
# env = Env(hps, data)
# obs = env.reset()
# obs.values(data, hps.days)

In [ ]:
env = Env(hps, data)
agent = Agent(hps, env, data)

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ./model/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0


In [ ]:
obs = env.reset()
rewards = []
reward_sum = 0
n = 100000
for i in range(n):
    print('{}/{}'.format(i, n))
    action = agent.step(obs)
    obs, reward, done = env.step(obs, action)
    rewards.append(reward)
    reward_sum += reward
    print('reward is {:.2f}, reward sum is {:.2f}, time is {}, close is {}\n'.format(
        reward, reward_sum, obs.decode(data, 1)['trade_time'].iloc[0], obs.decode(data, 1)['close'].iloc[0]))


0/100000
action_prob: [0.39657342 0.38204104 0.22138563], action is sell
reward is -0.00, reward sum is 0.00, time is 21.0, close is 28.28

1/100000
action_prob: [0.43359554 0.3283313  0.23807319], action is buy
reward is -0.00, reward sum is 0.00, time is 22.0, close is 28.27

2/100000
action_prob: [0.4135311  0.37282205 0.21364687], action is buy
reward is 8.00, reward sum is 8.00, time is 23.0, close is 28.35

3/100000
action_prob: [0.4195073  0.33264577 0.24784693], action is sell
reward is 2.00, reward sum is 10.00, time is 24.0, close is 28.37

4/100000
action_prob: [0.45859054 0.33935025 0.20205916], action is sell
reward is 5.00, reward sum is 15.00, time is 25.0, close is 28.42

5/100000
action_prob: [0.42529476 0.34749025 0.22721496], action is sell
reward is -8.00, reward sum is 7.00, time is 26.0, close is 28.34

6/100000
action_prob: [0.385062   0.39691684 0.21802115], action is buy
reward is 1.00, reward sum is 8.00, time is 27.0, close is 28.35

7/100000
action_prob: [0.

action_loss: -38.2979736328125, Q_loss : -22.382976531982422
reward is 11.00, reward sum is 137.85, time is 19.0, close is 30.21

48/100000
action_prob: [0.3357955  0.45067617 0.21352829], action is buy
action_loss: -38.7000617980957, Q_loss : 24.04739761352539
reward is -9.00, reward sum is 128.85, time is 20.0, close is 30.12

49/100000
action_prob: [0.33346924 0.45320922 0.2133215 ], action is hold
action_loss: -39.10194778442383, Q_loss : -5.391016960144043
reward is 8.00, reward sum is 136.85, time is 21.0, close is 30.2

50/100000
action_prob: [0.33111945 0.45573086 0.21314968], action is sell
action_loss: -39.50363540649414, Q_loss : 17.604965209960938
reward is -12.00, reward sum is 124.85, time is 22.0, close is 30.08

51/100000
action_prob: [0.32874912 0.4582401  0.21301083], action is sell
action_loss: -39.90513229370117, Q_loss : -5.981053829193115
reward is 11.00, reward sum is 135.85, time is 23.0, close is 30.19

52/100000
action_prob: [0.3263612  0.46073583 0.21290295],

action_loss: -54.64453125, Q_loss : 47.857948303222656
reward is -10.00, reward sum is 154.66, time is 11.0, close is 29.9

89/100000
action_prob: [0.25213972 0.531382   0.21647824], action is buy
action_loss: -55.0401611328125, Q_loss : 66.44960021972656
reward is 10.00, reward sum is 164.66, time is 12.0, close is 30.0

90/100000
action_prob: [0.25092894 0.53252316 0.2165479 ], action is sell
action_loss: -55.4356689453125, Q_loss : 90.44564819335938
reward is -6.00, reward sum is 158.66, time is 13.0, close is 29.94

91/100000
action_prob: [0.2497609  0.533627   0.21661212], action is sell
action_loss: -55.8310546875, Q_loss : 24.441688537597656
reward is -2.00, reward sum is 156.66, time is 14.0, close is 29.92

92/100000
action_prob: [0.24863449 0.53469443 0.21667105], action is sell
action_loss: -56.22632598876953, Q_loss : -2.1442646980285645
reward is -9.00, reward sum is 147.66, time is 15.0, close is 29.83

93/100000
action_prob: [0.24754857 0.53572655 0.2167249 ], action is 

action_loss: -70.79210662841797, Q_loss : -26.707923889160156
reward is 0.00, reward sum is 44.07, time is 3.0, close is 28.83

130/100000
action_prob: [0.226019   0.5573627  0.21661833], action is sell
action_loss: -71.18463134765625, Q_loss : -7.307242393493652
reward is 0.00, reward sum is 44.07, time is 4.0, close is 28.96

131/100000
action_prob: [0.22573732 0.5576723  0.21659033], action is sell
action_loss: -71.57711791992188, Q_loss : 20.702228546142578
reward is 0.00, reward sum is 44.07, time is 5.0, close is 28.99

132/100000
action_prob: [0.22546421 0.55797374 0.21656206], action is sell
action_loss: -71.96955871582031, Q_loss : -57.71969985961914
reward is 0.00, reward sum is 44.07, time is 6.0, close is 29.0

133/100000
action_prob: [0.22519927 0.55826724 0.21653353], action is buy
action_loss: -72.36196899414062, Q_loss : 14.27638053894043
reward is -0.58, reward sum is 43.49, time is 7.0, close is 29.0

134/100000
action_prob: [0.22494228 0.5585529  0.2165048 ], action 

action_loss: -86.85971069335938, Q_loss : -0.8685989379882812
reward is 16.00, reward sum is -32.51, time is 44.0, close is 28.24

171/100000
action_prob: [0.21901953 0.56550527 0.21547516], action is sell
action_loss: -87.25111389160156, Q_loss : 35.12749481201172
reward is 16.00, reward sum is -16.51, time is 45.0, close is 28.4

172/100000
action_prob: [0.2189215  0.565628   0.21545058], action is sell
action_loss: -87.64248657226562, Q_loss : -89.87643432617188
reward is -19.00, reward sum is -35.51, time is 46.0, close is 28.21

173/100000
action_prob: [0.2188255  0.56574833 0.21542618], action is buy
action_loss: -88.03385925292969, Q_loss : -12.88033676147461
reward is -12.00, reward sum is -47.51, time is 47.0, close is 28.09

174/100000
action_prob: [0.21873154 0.5658665  0.215402  ], action is buy
action_loss: -88.42520141601562, Q_loss : 32.115753173828125
reward is 25.00, reward sum is -22.51, time is 48.0, close is 28.34

175/100000
action_prob: [0.21863951 0.56598246 0.21

...

action_loss: -2729.41015625, Q_loss : -28.304006576538086
reward is -10.96, reward sum is 3114.97, time is 8.0, close is 61.449

6946/100000
action_prob: [0.21194291 0.5761124  0.21194474], action is hold
action_loss: -2729.7998046875, Q_loss : -13.388256072998047
reward is 10.96, reward sum is 3125.93, time is 9.0, close is 61.5586

6947/100000
action_prob: [0.21194291 0.5761124  0.21194474], action is sell
action_loss: -2730.190185546875, Q_loss : 26.508058547973633
reward is -17.93, reward sum is 3108.00, time is 10.0, close is 61.3793

6948/100000
action_prob: [0.21194291 0.5761124  0.21194474], action is hold
action_loss: -2730.580078125, Q_loss : -68.19590759277344
reward is 6.97, reward sum is 3114.97, time is 11.0, close is 61.449

6949/100000
action_prob: [0.21194291 0.5761124  0.21194474], action is sell
action_loss: -2730.97021484375, Q_loss : -150.83804321289062
reward is -16.93, reward sum is 3098.04, time is 12.0, close is 61.2797

6950/100000
action_prob: [0.21194291 0.5

action_loss: -2745.01123046875, Q_loss : 23.27004623413086
reward is -13.94, reward sum is 3027.32, time is 48.0, close is 60.5725

6986/100000
action_prob: [0.21194287 0.5761124  0.21194467], action is sell
action_loss: -2745.4013671875, Q_loss : -64.263916015625
reward is -20.92, reward sum is 3006.40, time is 0.0, close is 60.3633

6987/100000
action_prob: [0.21194287 0.5761124  0.21194467], action is sell
action_loss: -2745.791259765625, Q_loss : -93.1878890991211
reward is -41.08, reward sum is 2965.32, time is 1.0, close is 59.9648

6988/100000
action_prob: [0.21194287 0.57611245 0.21194468], action is sell
action_loss: -2746.181640625, Q_loss : 43.99507141113281
reward is 0.00, reward sum is 2965.32, time is 2.0, close is 60.0645

6989/100000
action_prob: [0.21194288 0.5761125  0.21194468], action is sell
action_loss: -2746.5712890625, Q_loss : -4.535820960998535
reward is 0.00, reward sum is 2965.32, time is 3.0, close is 59.5166

6990/100000
action_prob: [0.21194288 0.5761125 

action_loss: -2760.61279296875, Q_loss : 24.101402282714844
reward is -9.97, reward sum is 2854.53, time is 39.0, close is 58.5105

7026/100000
action_prob: [0.21194285 0.57611257 0.21194462], action is sell
action_loss: -2761.0029296875, Q_loss : 6.260135650634766
reward is 10.96, reward sum is 2865.49, time is 40.0, close is 58.6201

7027/100000
action_prob: [0.21194285 0.57611257 0.21194462], action is buy
action_loss: -2761.392822265625, Q_loss : -82.24089813232422
reward is 4.98, reward sum is 2870.47, time is 41.0, close is 58.6699

7028/100000
action_prob: [0.21194285 0.57611257 0.21194462], action is sell
action_loss: -2761.78271484375, Q_loss : 22.172203063964844
reward is 3.99, reward sum is 2874.46, time is 42.0, close is 58.7098

7029/100000
action_prob: [0.21194285 0.57611257 0.21194462], action is sell
action_loss: -2762.1728515625, Q_loss : -79.67608642578125
reward is -26.90, reward sum is 2847.56, time is 43.0, close is 58.4408

7030/100000
action_prob: [0.21194285 0.5

action_loss: -2776.21435546875, Q_loss : 18.057849884033203
reward is 16.94, reward sum is 2803.38, time is 30.0, close is 56.9268

7066/100000
action_prob: [0.21194282 0.5761126  0.21194455], action is hold
action_loss: -2776.60400390625, Q_loss : 64.80389404296875
reward is -44.83, reward sum is 2758.55, time is 31.0, close is 56.4785

7067/100000
action_prob: [0.21194282 0.5761126  0.21194454], action is sell
action_loss: -2776.994140625, Q_loss : -27.779773712158203
reward is -9.96, reward sum is 2748.59, time is 32.0, close is 56.3789

7068/100000
action_prob: [0.21194282 0.5761126  0.21194454], action is buy
action_loss: -2777.38427734375, Q_loss : -20.823741912841797
reward is -9.96, reward sum is 2738.63, time is 33.0, close is 56.2793

7069/100000
action_prob: [0.21194282 0.5761126  0.21194454], action is sell
action_loss: -2777.77392578125, Q_loss : 17.935630798339844
reward is 24.90, reward sum is 2763.53, time is 34.0, close is 56.5283

7070/100000
action_prob: [0.21194282 

action_loss: -2791.81591796875, Q_loss : 25.861907958984375
reward is 0.00, reward sum is 2760.28, time is 21.0, close is 56.5184

7106/100000
action_prob: [0.2119428  0.57611275 0.21194449], action is sell
action_loss: -2792.20556640625, Q_loss : -28.782054901123047
reward is 0.00, reward sum is 2760.28, time is 22.0, close is 56.5184

7107/100000
action_prob: [0.21194279 0.57611275 0.21194449], action is sell
action_loss: -2792.59521484375, Q_loss : -68.7960205078125
reward is 0.00, reward sum is 2760.28, time is 23.0, close is 56.5184

7108/100000
action_prob: [0.21194279 0.57611275 0.21194449], action is hold
action_loss: -2792.985595703125, Q_loss : 9.850322723388672
reward is 0.00, reward sum is 2760.28, time is 24.0, close is 56.5184

7109/100000
action_prob: [0.21194279 0.57611275 0.21194449], action is buy
action_loss: -2793.37548828125, Q_loss : -62.84365463256836
reward is 0.00, reward sum is 2760.28, time is 25.0, close is 56.5184

7110/100000
action_prob: [0.21194279 0.576

In [ ]:
agent._data_set._buffer